In [1]:
import os
import glob
import torch
import torch.nn.functional as F
import nbis
import numpy as np
import cv2
import torchvision.transforms as T
from PIL import Image
from tqdm import tqdm

In [2]:
from model.gumnet import GumNet
from model.alternate.gumnet_ap import GumNet as GumNetAP
from model.alternate.gumnet_mp import GumNet as GumNetMP

In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
CHECKPOINT_PATH = './checkpoints/gumnet_2d_best_noise_level_0_8x8_200.pth'
DATA_DIR = "./data"
GRID_SIZE = 8

settings = nbis.NbisExtractorSettings(
    min_quality=0.0,
    get_center=False,
    check_fingerprint=False,
    compute_nfiq2=False,
    ppi=500
)
extractor = nbis.new_nbis_extractor(settings)

## Native Resolution Image Warping
The function `warp_native_resolution()` implements a high-resolution Spatial Transformer Network (STN) approach. It applies the deformation learned by Gum-Net at a lower resolution (e.g., $192 \times 192$) to the original, high-resolution fingerprint image.Mathematical Formulation:Let $\Phi \in \mathbb{R}^{2 \times G \times G}$ be the control point grid predicted by the model, where $G$ is the `GRID_SIZE`.Grid Upsampling: The control points are interpolated to the native resolution $(H_n, W_n)$ using bicubic interpolation:$$\Delta = \text{Bicubic}(\Phi, (H_n, W_n))$$where $\Delta$ represents the dense flow field.Base Grid Generation: A standardized identity grid $\mathcal{G}$ is created such that for each pixel $(x, y)$, the coordinates are normalized to $[-1, 1]$:$$\mathcal{G}_{x,y} = \left( \frac{2x}{W_n - 1} - 1, \frac{2y}{H_n - 1} - 1 \right)$$Deformation: The final sampling grid $\mathcal{T}$ is the sum of the identity grid and the predicted flow:$$\mathcal{T} = \mathcal{G} + \Delta$$ Sampling: The warped image $I_{warped}$ is produced via bilinear sampling of the original image $I_{native}$ at positions specified by $\mathcal{T}$:$$I_{warped}(x, y) = \text{Sample}(I_{native}, \mathcal{T}_{x,y})$$

In [4]:
def warp_native_resolution(control_points, native_image_tensor):
    """
    Applies the GumNet control points to the pristine, native-resolution image.
    """
    B, C, H_native, W_native = native_image_tensor.size()
    device = control_points.device
    
    # 1. Upsample the control point grid to match native resolution
    dense_flow_native = F.interpolate(
        control_points, 
        size=(H_native, W_native), 
        mode='bicubic', 
        align_corners=True
    ) 
    
    # Permute to match grid_sample format: [B, H, W, 2]
    dense_flow_native = dense_flow_native.permute(0, 2, 3, 1)
    
    # 2. Create the base geometric grid at native resolution
    y, x = torch.meshgrid(
        torch.linspace(-1.0, 1.0, H_native, device=device),
        torch.linspace(-1.0, 1.0, W_native, device=device),
        indexing='ij'
    )
    base_grid_native = torch.stack([x, y], dim=-1).unsqueeze(0).expand(B, -1, -1, -1)
    
    # 3. Add the dense flow to calculate the final deformation field
    deformation_grid = base_grid_native + dense_flow_native
    
    # 4. Apply Spatial Transformer to the pristine native image
    warped_native_image = F.grid_sample(
        native_image_tensor, 
        deformation_grid, 
        mode='bilinear', 
        padding_mode='border', 
        align_corners=True
    )
    
    return warped_native_image

In [5]:
def construct_evaluation_pairs(base_data_dir, noise_level="Noise_Level_0"):
    """Scans the directory structure to construct genuine and imposter pairs."""
    classes = [
        "Arch_Only", "Double_Loop_Only", "Left_Loop_Only", "Natural",
        "Right_Loop_Only", "Tented_Arch_Only", "Whorl_Only"
    ]
    genuine_pairs, imposter_pairs = [], []
    class_probes = {}
    
    for cls in classes:
        probe_dir = os.path.join(base_data_dir, cls, noise_level, "test")
        class_probes[cls] = glob.glob(os.path.join(probe_dir, "*.png"))

    for master_cls in classes:
        master_template_path = os.path.join(base_data_dir, master_cls, "master", "1.png")
        if not os.path.exists(master_template_path): continue

        for probe_cls in classes:
            for probe_path in class_probes[probe_cls]:
                pair = (master_template_path, probe_path)
                if master_cls == probe_cls:
                    genuine_pairs.append(pair)
                else:
                    imposter_pairs.append(pair)
                    
    return genuine_pairs, imposter_pairs


def load_dual_tensors(image_path):
    """
    Returns TWO tensors:
    1. A 192x192 tensor for the GumNet forward pass.
    2. A native resolution tensor for the high-res warping function.
    """
    img = Image.open(image_path)
    
    # Base transforms (Tensor conversion and inversion needed by your model)
    to_tensor_and_invert = T.Compose([
        T.ToTensor(),
        T.RandomInvert(p=1.0) 
    ])
    
    # 1. Native Tensor (No Resizing)
    native_tensor = to_tensor_and_invert(T.Grayscale()(img)).unsqueeze(0).to(DEVICE)
    
    # 2. Model Tensor (Resized to 192x192)
    model_tensor = T.Resize((192, 192))(native_tensor)
    
    return model_tensor, native_tensor

In [6]:
def tensor_to_image_bytes(tensor):
    """
    Converts the native warped tensor back to standard PNG bytes for NBIS.
    """
    img_array = tensor.squeeze().detach().cpu().numpy()
    img_array = (img_array * 255).clip(0, 255).astype(np.uint8)
    
    # Revert the RandomInvert(p=1.0) so NBIS sees black ridges on white background
    img_array = cv2.bitwise_not(img_array)
    
    is_success, buffer = cv2.imencode(".png", img_array)
    return buffer.tobytes()

def compute_match_score(img_bytes_1, img_bytes_2):
    """Extracts minutiae and computes the NBIS Bozorth3 similarity score."""
    try:
        minutiae_1 = extractor.extract_minutiae(img_bytes_1)
        minutiae_2 = extractor.extract_minutiae(img_bytes_2)
        return minutiae_1.compare(minutiae_2)
    except Exception as e:
        return 0.0

In [7]:
def evaluate_pipeline(model, pairs, desc="Evaluating"):
    baseline_scores = []
    aligned_scores = []
    
    model.eval()
        
    for template_path, probe_path in tqdm(pairs, desc=desc):
        
        # --- BASELINE PIPELINE (Raw Disk Images) ---
        template_bytes = open(template_path, "rb").read()
        probe_bytes = open(probe_path, "rb").read()
        b_score = compute_match_score(template_bytes, probe_bytes)
        baseline_scores.append(b_score)
        
        # --- GUMNET ALIGNED PIPELINE (High-Res) ---
        with torch.no_grad():
            template_model, _ = load_dual_tensors(template_path)
            probe_model, probe_native = load_dual_tensors(probe_path)
            
            # 1. Forward pass on 192x192 to get control points
            _, control_points = model(template_model, probe_model)
            
            # 2. Warp the high-res native impression using the control points
            warped_native_probe = warp_native_resolution(control_points, probe_native)
            
        # Convert the high-res warped tensor back to image bytes
        warped_probe_bytes = tensor_to_image_bytes(warped_native_probe)
        
        a_score = compute_match_score(template_bytes, warped_probe_bytes)
        aligned_scores.append(a_score)

    return np.array(baseline_scores), np.array(aligned_scores)

In [8]:
print("Constructing evaluation pairs...")
genuine_pairs, imposter_pairs = construct_evaluation_pairs(DATA_DIR, noise_level="Noise_Level_0")

print("Loading GumNet model...")
gumnet_model = GumNet(in_channels=1, grid_size=GRID_SIZE).to(DEVICE)
if os.path.exists(CHECKPOINT_PATH):
    gumnet_model.load_state_dict(torch.load(CHECKPOINT_PATH, map_location=DEVICE))
    print(f"Successfully loaded weights from {CHECKPOINT_PATH}")
else:
    print(f"ERROR: Checkpoint {CHECKPOINT_PATH} not found!")

print("\nRunning Genuine Pairs...")
gen_base, gen_aligned = evaluate_pipeline(gumnet_model, genuine_pairs, desc="Genuine")

print("\nRunning Imposter Pairs...")
imp_base, imp_aligned = evaluate_pipeline(gumnet_model, imposter_pairs, desc="Imposter")

print("\n--- RESULTS ---")
print(f"Average Baseline Genuine Score: {gen_base.mean():.2f}")
print(f"Average Aligned Genuine Score:  {gen_aligned.mean():.2f}")
print("----------------")
print(f"Average Baseline Imposter Score: {imp_base.mean():.2f}")
print(f"Average Aligned Imposter Score:  {imp_aligned.mean():.2f}")

Constructing evaluation pairs...
Loading GumNet model...
Successfully loaded weights from ./checkpoints/gumnet_2d_best_noise_level_0_8x8_200.pth

Running Genuine Pairs...


Genuine: 100%|██████████| 700/700 [02:56<00:00,  3.97it/s]



Running Imposter Pairs...


Imposter: 100%|██████████| 4200/4200 [08:51<00:00,  7.90it/s]


--- RESULTS ---
Average Baseline Genuine Score: 153.25
Average Aligned Genuine Score:  170.39
----------------
Average Baseline Imposter Score: 10.06
Average Aligned Imposter Score:  10.25


## Decidability Index ($d'$)

The notebook concludes by calculating metrics to quantify the improvement in fingerprint matching after alignment.Decidability Index ($d'$)The decidability index measures the separation between the distributions of Genuine and Imposter matching scores. It is calculated as:$$d' = \frac{|\mu_{gen} - \mu_{imp}|}{\sqrt{\frac{1}{2}(\sigma^2_{gen} + \sigma^2_{imp})}}$$

In [9]:
def calculate_d_prime(gen_scores, imp_scores):
    """
    Calculates the Decidability Index (d') between two score distributions.
    """
    mu_gen = np.mean(gen_scores)
    mu_imp = np.mean(imp_scores)
    var_gen = np.var(gen_scores, ddof=1)
    var_imp = np.var(imp_scores, ddof=1)
    numerator = abs(mu_gen - mu_imp)
    denominator = np.sqrt(0.5 * (var_gen + var_imp))
    if denominator == 0:
        return 0.0
        
    return numerator / denominator

In [10]:
d_prime_base = calculate_d_prime(gen_base, imp_base)
d_prime_aligned = calculate_d_prime(gen_aligned, imp_aligned)

print("\n" + "="*70)
print(f"{'BIOMETRIC MATCHING PERFORMANCE METRICS':^70}")
print("="*70)
print(f"{'Metric':<25} | {'Baseline':<12} | {'GumNet-2D':<12} | {'Delta':<10}")
print("-" * 70)
print(f"{'Genuine Mean (μ_gen)':<25} | {gen_base.mean():<12.2f} | {gen_aligned.mean():<12.2f} | {(gen_aligned.mean() - gen_base.mean()):+.2f}")
print(f"{'Imposter Mean (μ_imp)':<25} | {imp_base.mean():<12.2f} | {imp_aligned.mean():<12.2f} | {(imp_aligned.mean() - imp_base.mean()):+.2f}")
print("-" * 70)
d_prime_label = "Decidability Index (d')"
print(f"{d_prime_label:<25} | {d_prime_base:<12.4f} | {d_prime_aligned:<12.4f} | {(d_prime_aligned - d_prime_base):+.4f}")
print("="*70)


                BIOMETRIC MATCHING PERFORMANCE METRICS                
Metric                    | Baseline     | GumNet-2D    | Delta     
----------------------------------------------------------------------
Genuine Mean (μ_gen)      | 153.25       | 170.39       | +17.14
Imposter Mean (μ_imp)     | 10.06        | 10.25        | +0.19
----------------------------------------------------------------------
Decidability Index (d')   | 4.0610       | 4.8739       | +0.8128


In [11]:
CHECKPOINT_PATH = './checkpoints/gumnetap_2d_best_noise_level_0_8x8_200.pth'

print("Constructing evaluation pairs...")
genuine_pairs, imposter_pairs = construct_evaluation_pairs(DATA_DIR, noise_level="Noise_Level_0")

print("Loading GumNet model...")
gumnet_model = GumNetAP(in_channels=1, grid_size=GRID_SIZE).to(DEVICE)
if os.path.exists(CHECKPOINT_PATH):
    gumnet_model.load_state_dict(torch.load(CHECKPOINT_PATH, map_location=DEVICE))
    print(f"Successfully loaded weights from {CHECKPOINT_PATH}")
else:
    print(f"ERROR: Checkpoint {CHECKPOINT_PATH} not found!")

print("\nRunning Genuine Pairs...")
gen_base, gen_aligned = evaluate_pipeline(gumnet_model, genuine_pairs, desc="Genuine")

print("\nRunning Imposter Pairs...")
imp_base, imp_aligned = evaluate_pipeline(gumnet_model, imposter_pairs, desc="Imposter")

print("\n--- RESULTS ---")
print(f"Average Baseline Genuine Score: {gen_base.mean():.2f}")
print(f"Average Aligned Genuine Score:  {gen_aligned.mean():.2f}")
print("----------------")
print(f"Average Baseline Imposter Score: {imp_base.mean():.2f}")
print(f"Average Aligned Imposter Score:  {imp_aligned.mean():.2f}")

Constructing evaluation pairs...
Loading GumNet model...
Successfully loaded weights from ./checkpoints/gumnetap_2d_best_noise_level_0_8x8_200.pth

Running Genuine Pairs...


Genuine: 100%|██████████| 700/700 [03:02<00:00,  3.84it/s]



Running Imposter Pairs...


Imposter: 100%|██████████| 4200/4200 [08:49<00:00,  7.94it/s]


--- RESULTS ---
Average Baseline Genuine Score: 153.25
Average Aligned Genuine Score:  152.65
----------------
Average Baseline Imposter Score: 10.06
Average Aligned Imposter Score:  10.30


In [12]:
d_prime_base = calculate_d_prime(gen_base, imp_base)
d_prime_aligned = calculate_d_prime(gen_aligned, imp_aligned)

print("\n" + "="*70)
print(f"{'BIOMETRIC MATCHING PERFORMANCE METRICS':^70}")
print("="*70)
print(f"{'Metric':<25} | {'Baseline':<12} | {'GumNet-2D-AP':<12} | {'Delta':<10}")
print("-" * 70)
print(f"{'Genuine Mean (μ_gen)':<25} | {gen_base.mean():<12.2f} | {gen_aligned.mean():<12.2f} | {(gen_aligned.mean() - gen_base.mean()):+.2f}")
print(f"{'Imposter Mean (μ_imp)':<25} | {imp_base.mean():<12.2f} | {imp_aligned.mean():<12.2f} | {(imp_aligned.mean() - imp_base.mean()):+.2f}")
print("-" * 70)
d_prime_label = "Decidability Index (d')"
print(f"{d_prime_label:<25} | {d_prime_base:<12.4f} | {d_prime_aligned:<12.4f} | {(d_prime_aligned - d_prime_base):+.4f}")
print("="*70)


                BIOMETRIC MATCHING PERFORMANCE METRICS                
Metric                    | Baseline     | GumNet-2D-AP | Delta     
----------------------------------------------------------------------
Genuine Mean (μ_gen)      | 153.25       | 152.65       | -0.60
Imposter Mean (μ_imp)     | 10.06        | 10.30        | +0.24
----------------------------------------------------------------------
Decidability Index (d')   | 4.0610       | 3.7788       | -0.2823


In [13]:
CHECKPOINT_PATH = './checkpoints/gumnetmp_2d_best_noise_level_0_8x8_200.pth'

print("Constructing evaluation pairs...")
genuine_pairs, imposter_pairs = construct_evaluation_pairs(DATA_DIR, noise_level="Noise_Level_0")

print("Loading GumNet model...")
gumnet_model = GumNetAP(in_channels=1, grid_size=GRID_SIZE).to(DEVICE)
if os.path.exists(CHECKPOINT_PATH):
    gumnet_model.load_state_dict(torch.load(CHECKPOINT_PATH, map_location=DEVICE))
    print(f"Successfully loaded weights from {CHECKPOINT_PATH}")
else:
    print(f"ERROR: Checkpoint {CHECKPOINT_PATH} not found!")

print("\nRunning Genuine Pairs...")
gen_base, gen_aligned = evaluate_pipeline(gumnet_model, genuine_pairs, desc="Genuine")

print("\nRunning Imposter Pairs...")
imp_base, imp_aligned = evaluate_pipeline(gumnet_model, imposter_pairs, desc="Imposter")

print("\n--- RESULTS ---")
print(f"Average Baseline Genuine Score: {gen_base.mean():.2f}")
print(f"Average Aligned Genuine Score:  {gen_aligned.mean():.2f}")
print("----------------")
print(f"Average Baseline Imposter Score: {imp_base.mean():.2f}")
print(f"Average Aligned Imposter Score:  {imp_aligned.mean():.2f}")

Constructing evaluation pairs...
Loading GumNet model...
Successfully loaded weights from ./checkpoints/gumnetmp_2d_best_noise_level_0_8x8_200.pth

Running Genuine Pairs...


Genuine: 100%|██████████| 700/700 [02:48<00:00,  4.16it/s]



Running Imposter Pairs...


Imposter: 100%|██████████| 4200/4200 [08:35<00:00,  8.15it/s]


--- RESULTS ---
Average Baseline Genuine Score: 153.25
Average Aligned Genuine Score:  123.75
----------------
Average Baseline Imposter Score: 10.06
Average Aligned Imposter Score:  9.39


In [14]:
d_prime_base = calculate_d_prime(gen_base, imp_base)
d_prime_aligned = calculate_d_prime(gen_aligned, imp_aligned)

print("\n" + "="*70)
print(f"{'BIOMETRIC MATCHING PERFORMANCE METRICS':^70}")
print("="*70)
print(f"{'Metric':<25} | {'Baseline':<12} | {'Gum-Net-MP-2D':<12} | {'Delta':<10}")
print("-" * 70)
print(f"{'Genuine Mean (μ_gen)':<25} | {gen_base.mean():<12.2f} | {gen_aligned.mean():<12.2f} | {(gen_aligned.mean() - gen_base.mean()):+.2f}")
print(f"{'Imposter Mean (μ_imp)':<25} | {imp_base.mean():<12.2f} | {imp_aligned.mean():<12.2f} | {(imp_aligned.mean() - imp_base.mean()):+.2f}")
print("-" * 70)
d_prime_label = "Decidability Index (d')"
print(f"{d_prime_label:<25} | {d_prime_base:<12.4f} | {d_prime_aligned:<12.4f} | {(d_prime_aligned - d_prime_base):+.4f}")


                BIOMETRIC MATCHING PERFORMANCE METRICS                
Metric                    | Baseline     | Gum-Net-MP-2D | Delta     
----------------------------------------------------------------------
Genuine Mean (μ_gen)      | 153.25       | 123.75       | -29.49
Imposter Mean (μ_imp)     | 10.06        | 9.39         | -0.67
----------------------------------------------------------------------
Decidability Index (d')   | 4.0610       | 3.4025       | -0.6585
